# 2nd Level Model Structure: Reboiler

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.energy.energy_reboiler import energy_block_rule
from physics.VLE.VLE_reboiler_MPCC_P import VLE_block_rule
from physics.MPCC.MPCC_P import P_NCP_block_rule, P_Reg_block_rule, P_pf_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,350+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.Q_main = pe.Var(within=pe.Reals) # MW

# Construct Individual Blocks

In [5]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [6]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.P_VLE
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [7]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [8]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

# MPCC - P-Pspec

In [9]:
model.MPCC = pe.Block(rule = P_pf_block_rule)

> Importing MPCC_P_Reg Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC.s_L
| MPCC.s_V
| MPCC.pf
| MPCC.rho
--------------------------------------------------
> Spliting pressure used in VLE



## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [10]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [11]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [12]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Load a flash liquid composition as feed

In [13]:
with open('../saved_solutions/reactive_flash_MPCC_P_pf_200C.pickle', 'rb') as f:
    C200 = pickle.load(f)

## Fixing In/Product Redundent Flow

In [14]:
model.y_.fix(0)
for i in m.COMP_TOTAL:
    model.x_['in',i].fix(C200.Solution.Variable['x[{}]'.format(i)]['Value'])

model.L['in'].fix(1)
# model.L['in'].setlb(1)
# model.L['in'].setub(1e-8)

model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_V_.fix(0)
model.H_L_.fix(C200.Solution.Variable['H_L']['Value'])

## Fixing Model Parameters

In [15]:
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(0)
model.z.fix(0)
model.VLE_block.n_ave.fix(20)
# model.Q_main.fix(0)

# kappa

In [16]:
for i in model.component_objects(pe.Constraint, active=True):
    print(i.name)

mass_balance_main_con
VL_equil_con
summation_x_y_con
summation_total_mass_con
heat_balance_main_con
energy_block.dH_V_con
energy_block.H_V_con
energy_block.dH_vap_con
energy_block.dH_L_con
energy_block.H_L_con
energy_block.dH_F_con
energy_block.H_F_con
VLE_block.f_L_HENRY_con
VLE_block.Hen_con
VLE_block.Hen0_con
VLE_block.Hen_ref_con
VLE_block.Hen0_ref_con
VLE_block.f_L_NONHENRY_con
VLE_block.gamma_con
VLE_block.gamma_ref_con
VLE_block.n_ave_con
VLE_block.P_sat_con
VLE_block.P_sat_con2
VLE_block.P_sat_dY_inf_con
VLE_block.P_sat_dY0_con
VLE_block.f_V_con
VLE_block.V_L_nonHen_con
VLE_block.V_L_dY_inf_con
VLE_block.V_L_dY0_con
VLE_block.V_L_Hen_con
VLE_block.V_L_H2O_con
VLE_block.poynting_con
MPCC.penalty_con
MPCC.pressure_equal_con


In [17]:
model.magic = pe.Var(within=pe.NonNegativeReals,initialize=1)

In [18]:
all_expression = []
for i in model.component_data_objects(pe.Constraint, active=True):
    if i.name.startswith('summation_x_y_con') or i.name.startswith('mass_balance_main_con'):
        all_expression.append(i.body)
        i.deactivate()

In [19]:
model.replaced_constraint_list = pe.ConstraintList()
for i in all_expression:
    model.replaced_constraint_list.add(expr = i*model.magic==0);

In [20]:
# model.complementarity_con = pe.Constraint(expr = 2*model.big_beta == \
#                             model.L['in'] - model.big_epi + ((model.big_epi - model.L['in'])**2+1e-10)**0.5)
model.complementarity_con = pe.Constraint(expr = model.magic == model.L['in'])

## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [21]:
model.T.fix(200+273.15)

In [22]:
check_DOF(pe,model)

Active Equality Constraints:	 958
Active Inequality Constraints:	 1
Active Variables:		 1133
Fixed Variables:		 172
DOF:				 3


In [23]:
model.obj = pe.Objective(expr = model.T - model.MPCC.pf ,sense=pe.maximize)
# model.obj = pe.Objective(expr = - model.L['in'] - model.MPCC.pf ,sense=pe.maximize)
# model.obj = pe.Objective(expr = - model.L['in'] ,sense=pe.maximize)

In [24]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90 -4.6754943e+02 1.55e+00 3.34e+01  -1.0 1.22e+03    -  9.16e-02 1.63e-02h  3
  91 -4.6761368e+02 1.52e+00 5.16e+01  -1.0 8.30e+02    -  9.00e-02 2.30e-02h  3
  92 -4.6769271e+02 1.48e+00 7.19e+01  -1.0 5.37e+02    -  1.44e-01 2.55e-02h  3
  93 -4.6777436e+02 1.44e+00 1.07e+02  -1.0 3.64e+02    -  1.56e-01 2.85e-02h  3
  94 -4.6759692e+02 1.39e+00 2.19e+02  -1.0 2.53e+02    -  3.55e-01 3.15e-02h  3
  95 -4.6722989e+02 1.35e+00 2.51e+02  -1.0 1.80e+02    -  5.24e-02 3.09e-02h  3
  96 -4.6694554e+02 1.29e+00 4.59e+02  -1.0 1.35e+02    -  4.85e-01 4.23e-02h  3
  97 -4.6676201e+02 1.23e+00 7.40e+02  -1.0 9.49e+01    -  2.78e-01 4.88e-02h  3
  98 -4.6688277e+02 1.16e+00 3.02e+03  -1.0 6.74e+01    -  9.05e-01 5.81e-02h  3
  99 -4.6859811e+02 8.44e-01 5.06e+04  -1.0 4.95e+01    -  5.05e-01 2.71e-01w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100 -4.6921201e+02 8.09e-01

In [25]:
model.Q_main.value

1.8335655222504552e-05

In [26]:
model.magic.value

1.0

In [27]:
model.L['in'].value

1

In [28]:
model.T.value

473.15

In [29]:
# model.solutions.store_to(results)
# with open('../saved_solutions/reboiler_MPCC_P_pf_200C.pickle','wb') as f:
#     pickle.dump(results,f)

In [30]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

Component		Liquid: 1.0000			Vapor: 0.0000			log K
------------------------------------------------------------------------------------------------------------
H2         		 0.8837%			25.2923%			1.4567
CO         		 1.3400%			28.6939%			1.3307
CO2        		 3.7244%			31.8633%			0.9322
H2O        		 3.6023%			12.4040%			0.5370
C2H4       		 0.0174%			 0.1174%			0.8302
C3H6       		 0.0444%			 0.1351%			0.4830
C4H8       		 0.0456%			 0.1204%			0.4221
C5H10      		 0.0722%			 0.0931%			0.1104
C6H12      		 0.1262%			 0.0865%			-0.1641
C7H14      		 0.2078%			 0.0799%			-0.4151
C8H16      		 0.3264%			 0.0731%			-0.6499
C9H18      		 0.4920%			 0.0659%			-0.8729
C10H20     		 0.7120%			 0.0583%			-1.0869
C11H22     		 0.6286%			 0.0320%			-1.2938
C12H24     		 0.8302%			 0.0266%			-1.4951
C13H26     		 1.0411%			 0.0212%			-1.6918
C14H28     		 1.2375%			 0.0161%			-1.8847
C15H30     		 1.3970%			 0.0118%			-2.0743
C16H32     		 1.5062%			 0.0083%			-2.2611
C17H34     		 1.5634%			 0.0056%

# Iterative Solve for Data Analysis

In [31]:
update_dual(pe,model)

In [32]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [33]:
rf_data = {}
rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = [];
rf_data['f_V_CO'] = []; rf_data['f_V_H2'] = [];
    
Trange = np.arange(350+273.15,199+273.15,-5)

for Tub in Trange:
    model.T.setub(Tub)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    # print('-'*72)
    print('Solved, Solution T = {:.2f} K\t|\tV = {:.5f}\t|\tL = {:.5f}\t|'.format(model.T.value,model.V['out'].value,model.L['out'].value))
    # print('-'*72)
    rf_data['T'].append(model.T.value)
    rf_data['Q'].append(model.Q_main.value)
    rf_data['V'].append(model.V['out'].value)
    rf_data['L'].append(model.L['out'].value)    
    
    
    rf_data['y_H2O'].append(model.y['H2O'].value)
    rf_data['y_CO'].append(model.y['CO'].value)
    rf_data['y_H2'].append(model.y['H2'].value)
    rf_data['y_CO2'].append(model.y['CO2'].value)
    
    rf_data['f_V_CO'].append(model.f_V['CO'].value)
    rf_data['f_V_H2'].append(model.f_V['H2'].value)

Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00000	|	L = 1.00000	|
Solved, Solution T = 473.15 K	|	V = 0.00

# VLE Validation using AspenPlus

### Feed (Reactor Effluent)

In [34]:
print('Temperature (C): \t{:.2f}\t\t'.format(model.T.value-273.15))
print('Total FLow (kmol/s): \t{:.2f}\t\t'.format(model.L['out'].value+model.V['out'].value))
print('-'*72)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:.4%}'.format((model.x[i].value*model.L['out'].value\
        +model.y[i].value*model.V['out'].value)/(model.L['out'].value+model.V['out'].value)))

Temperature (C): 	200.00		
Total FLow (kmol/s): 	1.00		
------------------------------------------------------------------------
H2         		0.8838%
CO         		1.3401%
CO2        		3.7245%
H2O        		3.6023%
C2H4       		0.0174%
C3H6       		0.0444%
C4H8       		0.0456%
C5H10      		0.0722%
C6H12      		0.1262%
C7H14      		0.2078%
C8H16      		0.3264%
C9H18      		0.4920%
C10H20     		0.7120%
C11H22     		0.6286%
C12H24     		0.8302%
C13H26     		1.0411%
C14H28     		1.2375%
C15H30     		1.3970%
C16H32     		1.5062%
C17H34     		1.5634%
C18H36     		1.5753%
C19H38     		1.5530%
C20H40     		1.5073%
C1H4       		0.0164%
C2H6       		0.0107%
C3H8       		0.0114%
C4H10      		0.0162%
C5H12      		0.0471%
C6H14      		0.0815%
C7H16      		0.1332%
C8H18      		0.2078%
C9H20      		0.3113%
C10H22     		0.4475%
C11H24     		0.9951%
C12H26     		1.3036%
C13H28     		1.6207%
C14H30     		1.9099%
C15H32     		2.1391%
C16H34     		2.2908%
C17H36     		2.3649%
C18H38     		2.3734%
C19H40    

### Phase Separation

In [35]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

Component		Liquid: 1.0000			Vapor: 0.0000			log K
------------------------------------------------------------------------------------------------------------
H2         		 0.8837%			25.2925%			1.4567
CO         		 1.3400%			28.6940%			1.3307
CO2        		 3.7244%			31.8632%			0.9322
H2O        		 3.6023%			12.4039%			0.5370
C2H4       		 0.0174%			 0.1174%			0.8302
C3H6       		 0.0444%			 0.1351%			0.4830
C4H8       		 0.0456%			 0.1204%			0.4221
C5H10      		 0.0722%			 0.0931%			0.1104
C6H12      		 0.1262%			 0.0865%			-0.1641
C7H14      		 0.2078%			 0.0799%			-0.4151
C8H16      		 0.3264%			 0.0731%			-0.6499
C9H18      		 0.4920%			 0.0659%			-0.8729
C10H20     		 0.7120%			 0.0583%			-1.0869
C11H22     		 0.6286%			 0.0320%			-1.2938
C12H24     		 0.8302%			 0.0266%			-1.4951
C13H26     		 1.0411%			 0.0212%			-1.6918
C14H28     		 1.2375%			 0.0161%			-1.8847
C15H30     		 1.3970%			 0.0118%			-2.0743
C16H32     		 1.5062%			 0.0083%			-2.2611
C17H34     		 1.5634%			 0.0056%

In [36]:
list(zip(rf_data['T'],rf_data['Q']))

[(473.15, 1.594178721742492e-05),
 (473.15, 1.594178733021147e-05),
 (473.15, 1.5941787354175736e-05),
 (473.15, 1.5941787380160162e-05),
 (473.15, 1.594178735275552e-05),
 (473.15, 1.5941787327820922e-05),
 (473.15, 1.594178735150207e-05),
 (473.15, 1.5941787315247287e-05),
 (473.15, 1.594178734796502e-05),
 (473.15, 1.5941787370856064e-05),
 (473.15, 1.59417873361847e-05),
 (473.15, 1.5941787360999104e-05),
 (473.15, 1.5941787328549845e-05),
 (473.15, 1.5941787347272912e-05),
 (473.15, 1.5941787370557377e-05),
 (473.15, 1.5941787338262284e-05),
 (473.15, 1.5941787357019564e-05),
 (473.15, 1.594178737752792e-05),
 (473.15, 1.594178734829503e-05),
 (473.15, 1.594178737149071e-05),
 (473.15, 1.594178733488257e-05),
 (473.15, 1.5941787360295874e-05),
 (473.15, 1.594178733257772e-05),
 (473.15, 1.594178735453526e-05),
 (473.15, 1.5941787321891566e-05),
 (473.15, 1.594178733820964e-05),
 (473.15, 1.5941787362202813e-05),
 (473.15, 1.5941787328960148e-05),
 (473.15, 1.5941787348839435e-05),